# About

It's difficult to use functions in jupyter notebook, since we want different steps to be in different cells, so one of the main functions of this module is to emulate a function like scope of the variables - which get destroyed at the end of the experiment. Some extra magic is added to reclaim GPU and General RAM.

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from ipyexperiments import *

## Setup and preload

In [3]:
import numpy as np
import torch

In [4]:
def consume_gpu_ram(n): return torch.ones((n, n)).cuda()
def consume_cpu_ram(n): return np.ones((n, n))

`pytorch`'s CUDA machinery seems to require ~0.5GB GPU RAM for this particular card, and ~2GB of RAM upon its first use, and it's not shared between processes. So if you use pytorch w/ CUDA and you have the same GPU - its real capacity is 0.5GB smaller from the get going, and multiply that by the number of concurrent processes. Newer GPUs may consume easily up to 1.5GB in CUDA kernels

Because of that, in order to get the numbers right, it can be a good idea to pre-load it by allocating a tiny tensor on `cuda`. If we don't - the first experiment' stats will be misleading/incorrect.

In [5]:
#z = consume_gpu_ram(1)

But, if you use `IPyExperiments` - it performs this preloading for you already, when the backend is loaded (see below).

## Experiment with no GPU

Let's consume a big chunk of non-GPU RAM and reclaim it at the end of the experiment.

In this experiment we use the `cpu` backend, so GPU RAM will not be managed, regardless of whether there is a GPU that can be used or not. This mode is primarily used for configurations without GPU.

In [6]:
exp1 = IPyExperimentsCPU()


*** Experiment started with the CPU-only backend


*** Current state:
RAM:    Used    Free   Total       Util
CPU:     163  15,127  31,588 MB   0.52% 


･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.000
･ CPU:          0          0        163 MB |


In [7]:
x1 = consume_cpu_ram(2**14) # about 2GB

･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.501
･ CPU:      2,048          0      2,210 MB |


In [8]:
x2 = consume_cpu_ram(2**14) # about 2GB

･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.467
･ CPU:      2,048          0      4,258 MB |


In [9]:
del exp1 # finish experiment

･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.000
･ CPU:          0          0      4,258 MB |

IPyExperimentsCPU: Finishing

*** Experiment finished in 00:00:01 (elapsed wallclock time)

*** Newly defined local variables:
Deleted: x1, x2

*** Experiment memory:
RAM: Consumed       Reclaimed
CPU:    4,095    4,095 MB (100.00%)

*** Current state:
RAM:    Used    Free   Total       Util
CPU:     163  15,121  31,588 MB   0.52% 




## GPU Experiment: consume general and GPU RAM

Let's consume a big chunk of each, general and GPU RAM and reclaim both of them, at the end of the experiment.

This time we wil use the GPU backed `pytorch`, so both GPU and general RAM will be managed. This is the default backed, so if you don't pass this argument, it'll default to `pytorch`.

In [10]:
exp2 = IPyExperimentsPytorch()


*** Experiment started with the Pytorch backend
Device: ID 0, GeForce GTX 1070 Ti (8119 RAM)


*** Current state:
RAM:    Used    Free   Total       Util
CPU:   1,780  13,667  31,588 MB   5.64% 
GPU:   2,807   5,312   8,119 MB  34.57% 


･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.000
･ CPU:          0          0      1,780 MB |
･ GPU:          0          0      2,807 MB |


In [11]:
x1 = consume_cpu_ram(2**14) # about 2GB

･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.469
･ CPU:      2,048          0      3,829 MB |
･ GPU:          0          0      2,807 MB |


In [12]:
x2 = consume_gpu_ram(2**14) # about 1GB

･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.173
･ CPU:          0          0      3,828 MB |
･ GPU:      1,024          0      3,831 MB |


In [13]:
del exp2 # finish experiment

･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.000
･ CPU:          0          0      3,828 MB |
･ GPU:          0          0      3,831 MB |

IPyExperimentsPytorch: Finishing

*** Experiment finished in 00:00:01 (elapsed wallclock time)

*** Newly defined local variables:
Deleted: x1, x2

*** Experiment memory:
RAM: Consumed       Reclaimed
CPU:    2,048    2,048 MB ( 99.99%)
GPU:    1,024    1,024 MB (100.00%)

*** Current state:
RAM:    Used    Free   Total       Util
CPU:   1,780  13,663  31,588 MB   5.64% 
GPU:   2,807   5,312   8,119 MB  34.57% 




## Get experiment data, preserve some vars

Here we demonstate features that help with using this framework programmatically. i.e. getting the functions to return experiment data during and at the end of the experiment, rather than just printing it. You can then use it to programmatically refine the hyper parameters before rerunning the experiment.

This experiment also demonstrates how to save some of the local variables.

In [14]:
exp3 = IPyExperimentsPytorch()


*** Experiment started with the Pytorch backend
Device: ID 0, GeForce GTX 1070 Ti (8119 RAM)


*** Current state:
RAM:    Used    Free   Total       Util
CPU:   1,780  13,667  31,588 MB   5.64% 
GPU:   2,807   5,312   8,119 MB  34.57% 


･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.000
･ CPU:          0          0      1,780 MB |
･ GPU:          0          0      2,807 MB |


In [15]:
x1 = consume_cpu_ram(2**14) # about 2GB

･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.471
･ CPU:      2,048          0      3,828 MB |
･ GPU:          0          0      2,807 MB |


Run an intermediary report of how much of the resources was consumed, and how much is available, returning the data as numbers. (none would be reclaimed yet, so it'll be zeros, but the return value is there for consistency).

In [16]:
cpu_data = exp3.data.cpu
gpu_data = exp3.data.gpu
print(cpu_data, gpu_data)

IPyExperimentMemory(consumed=2147278848, reclaimed=0, available=12177063936) IPyExperimentMemory(consumed=0, reclaimed=0, available=5570625536)
･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.001
･ CPU:          0          0      3,828 MB |
･ GPU:          0          0      2,807 MB |


Let's preserve these variables, so that they remain available after the experiment is finished and the rest of the local variables get deleted. 

Note, that you need to pass the names of the variables and not the variables themselves.

In [17]:
exp3.keep_var_names('cpu_data', 'gpu_data')

･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.020
･ CPU:          0          0      3,828 MB |
･ GPU:          0          0      2,807 MB |


In [18]:
x2 = consume_gpu_ram(2**14) # about 1GB

･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.202
･ CPU:          0          0      3,828 MB |
･ GPU:      1,024          0      3,831 MB |


Run another intermediary report.

In [19]:
cpu_data = exp3.data.cpu
gpu_data = exp3.data.gpu
print(cpu_data)
print(gpu_data)
print(cpu_data.consumed)
print(gpu_data.available)

IPyExperimentMemory(consumed=2147692544, reclaimed=0, available=12176052224)
IPyExperimentMemory(consumed=1073741824, reclaimed=0, available=4496883712)
2147692544
4496883712
･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.002
･ CPU:          0          0      3,828 MB |
･ GPU:          0          0      3,831 MB |


Complete the experiment, delete local vars, reclaim memory, and run the final report of how much of the resources was consumed, and how much is available, and how much was reclaimed, returning the data as numbers.

In [20]:
data = exp3.finish() # finish experiment
# or:
# _ = exp3.finish()
# data = exp3.data
cpu_data_final = data.cpu
gpu_data_final= data.gpu

print("\nNumerical data:\n", cpu_data_final, gpu_data_final)

･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.020
･ CPU:          0          0      3,828 MB |
･ GPU:          0          0      3,831 MB |

IPyExperimentsPytorch: Finishing

*** Experiment finished in 00:00:00 (elapsed wallclock time)

*** Newly defined local variables:
Deleted: x1, x2
Kept:    cpu_data, gpu_data

*** Experiment memory:
RAM: Consumed       Reclaimed
CPU:    2,048    2,047 MB ( 99.99%)
GPU:    1,024    1,024 MB (100.00%)

*** Current state:
RAM:    Used    Free   Total       Util
CPU:   1,780  13,661  31,588 MB   5.64% 
GPU:   2,807   5,312   8,119 MB  34.57% 



Numerical data:
 IPyExperimentMemory(consumed=2147692544, reclaimed=2147471360, available=14325116928) IPyExperimentMemory(consumed=1073741824, reclaimed=1073741824, available=5570625536)


And let's test that we can still access the variables we asked not to destroy:

In [21]:
print("\nHalf-way data:\n", cpu_data, gpu_data)


Half-way data:
 IPyExperimentMemory(consumed=2147692544, reclaimed=0, available=12176052224) IPyExperimentMemory(consumed=1073741824, reclaimed=0, available=4496883712)


## Using the context manager

If you want to put all cells into one, you could simplify the experiment even further by using its context manager.

In [22]:
with IPyExperimentsPytorch(): 
    x1 = consume_cpu_ram(2**14)
    x2 = consume_gpu_ram(2**14)


*** Experiment started with the Pytorch backend
Device: ID 0, GeForce GTX 1070 Ti (8119 RAM)


*** Current state:
RAM:    Used    Free   Total       Util
CPU:   1,780  13,665  31,588 MB   5.64% 
GPU:   2,807   5,312   8,119 MB  34.57% 


･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.645
･ CPU:      2,048          0      3,828 MB |
･ GPU:      1,024          0      3,831 MB |

IPyExperimentsPytorch: Finishing

*** Experiment finished in 00:00:00 (elapsed wallclock time)

*** Newly defined local variables:
Deleted: x1, x2

*** Experiment memory:
RAM: Consumed       Reclaimed
CPU:    2,048    2,048 MB (100.00%)
GPU:    1,024    1,024 MB (100.00%)

*** Current state:
RAM:    Used    Free   Total       Util
CPU:   1,780  13,660  31,588 MB   5.64% 
GPU:   2,807   5,312   8,119 MB  34.57% 




In [23]:
with IPyExperimentsPytorch() as exp: 
    x1 = consume_cpu_ram(2**14)
    z = "some data"
    x2 = consume_gpu_ram(2**14)
    exp.keep_var_names('z')
print(z)


*** Experiment started with the Pytorch backend
Device: ID 0, GeForce GTX 1070 Ti (8119 RAM)


*** Current state:
RAM:    Used    Free   Total       Util
CPU:   1,780  13,664  31,588 MB   5.64% 
GPU:   2,807   5,312   8,119 MB  34.57% 


･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.639
･ CPU:      2,048          0      3,828 MB |
･ GPU:      1,024          0      3,831 MB |

IPyExperimentsPytorch: Finishing

*** Experiment finished in 00:00:00 (elapsed wallclock time)

*** Newly defined local variables:
Deleted: x1, x2
Kept:    z

*** Experiment memory:
RAM: Consumed       Reclaimed
CPU:    2,048    2,048 MB (100.00%)
GPU:    1,024    1,024 MB (100.00%)

*** Current state:
RAM:    Used    Free   Total       Util
CPU:   1,780  13,661  31,588 MB   5.64% 
GPU:   2,807   5,312   8,119 MB  34.57% 


some data


In [24]:
%%javascript # prevent committing an unsaved notebook
IPython.notebook.save_notebook()

<IPython.core.display.Javascript object>